In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Load API Credentials
with open('/Users/Joene/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [10]:
# set our API call parameters 
LOCATION = 'Vancouver, Wa'
TERM = 'Pho'

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_vancouver_pho.json"
JSON_FILE

'Data/results_in_progress_vancouver_pho.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_vancouver_pho.json already exists.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [11]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [12]:
## How many results total?
total_results = results['total']
total_results

320

In [13]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [14]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

16

In [15]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [16]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/16 [00:00<?, ?it/s]

In [17]:
## delete file and confirm it no longer exits.
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [18]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [19]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_vancouver_pho.json not found. Saving empty list to new file.
- 0 previous results found.


16

In [20]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/16 [00:00<?, ?it/s]

In [21]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,JpiPjhEAQByKNgaHtK_FCA,pho-family-vancouver-2,Pho Family,https://s3-media2.fl.yelpcdn.com/bphoto/LVCTb0...,False,https://www.yelp.com/biz/pho-family-vancouver-...,29,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",5.0,"{'latitude': 45.617293, 'longitude': -122.555781}","[delivery, restaurant_reservation, pickup]",$$,"{'address1': '516 SE Chkalov Dr', 'address2': ...",+13602549305,(360) 254-9305,5402.031728
1,EfY85ruGuo6i8oo0v2dBTw,pho-ton-1-vancouver,Pho Ton 1,https://s3-media1.fl.yelpcdn.com/bphoto/tTGzD5...,False,https://www.yelp.com/biz/pho-ton-1-vancouver?a...,202,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.5,"{'latitude': 45.64345, 'longitude': -122.61035}","[delivery, pickup]",$$,"{'address1': '6000 NE 4th Plain Blvd', 'addres...",+13609935484,(360) 993-5484,1108.281651
2,wMtKDseD-2U39vae_Ym2Yg,pho-hong-phat-vancouver-2,Pho Hong Phat,https://s3-media2.fl.yelpcdn.com/bphoto/yGCEf5...,False,https://www.yelp.com/biz/pho-hong-phat-vancouv...,130,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,"{'latitude': 45.6380399, 'longitude': -122.63984}","[delivery, pickup]",$,"{'address1': '2904 E 4th Plain Blvd', 'address...",+13606938570,(360) 693-8570,1570.119314
3,j2nS1D5DjuJl5zcNkwPjQA,pho-divine-vancouver-2,Pho Divine,https://s3-media3.fl.yelpcdn.com/bphoto/p3OufC...,False,https://www.yelp.com/biz/pho-divine-vancouver-...,14,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",4.5,"{'latitude': 45.63718583664865, 'longitude': -...",[],$$,"{'address1': '2014 Main St', 'address2': '', '...",+13603144599,(360) 314-4599,4045.016622
4,nU-S7XRc14kT1wNM3rc_Yg,pho-star-in-vancouver-vancouver,Pho Star in Vancouver,https://s3-media2.fl.yelpcdn.com/bphoto/VY7om-...,False,https://www.yelp.com/biz/pho-star-in-vancouver...,147,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",4.5,"{'latitude': 45.62127, 'longitude': -122.56769}","[delivery, pickup]",$$,"{'address1': '10204 SE Mill Plain Blvd', 'addr...",+13602564002,(360) 256-4002,4369.686440


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
315,fsKFkh6_TBnhdTmXQuk-gg,nicholas-restaurant-portland-6,Nicholas Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/40ZpWL...,False,https://www.yelp.com/biz/nicholas-restaurant-p...,1332,"[{'alias': 'mediterranean', 'title': 'Mediterr...",4.0,"{'latitude': 45.51329547724797, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '1109 SE Madison St', 'address2':...",+15032355123,(503) 235-5123,13902.436516
316,88wLRYub7GQ70X1K1IICUA,migration-brewing-portland,Migration Brewing,https://s3-media1.fl.yelpcdn.com/bphoto/6TKk_D...,False,https://www.yelp.com/biz/migration-brewing-por...,312,"[{'alias': 'brewpubs', 'title': 'Brewpubs'}]",3.5,"{'latitude': 45.52623, 'longitude': -122.63635}","[delivery, pickup]",$$,"{'address1': '2828 NE Glisan St', 'address2': ...",+15032065221,(503) 206-5221,12272.069135
317,vBe0AOTcCn-EQmNksXtlew,fubonn-supermarket-portland,Fubonn Supermarket,https://s3-media4.fl.yelpcdn.com/bphoto/6B8aZR...,False,https://www.yelp.com/biz/fubonn-supermarket-po...,163,"[{'alias': 'intlgrocery', 'title': 'Internatio...",3.5,"{'latitude': 45.50165, 'longitude': -122.5768}",[],$$,"{'address1': '2850 SE 82nd Ave', 'address2': N...",+15035178877,(503) 517-8877,15285.058031
318,zkz1nksuZJwmtYeQTgLYUQ,toast-la-tea-portland,Toast La Tea,https://s3-media2.fl.yelpcdn.com/bphoto/zhlejF...,False,https://www.yelp.com/biz/toast-la-tea-portland...,97,"[{'alias': 'bubbletea', 'title': 'Bubble Tea'}...",3.5,"{'latitude': 45.5049705505371, 'longitude': -1...","[delivery, pickup]",$,"{'address1': '2446 SE 87th Ave', 'address2': '...",+19712797789,(971) 279-7789,14979.781683
319,VGn8veH_xI91nkcOAk7xiw,rose-city-liquor-portland-2,Rose City Liquor,https://s3-media1.fl.yelpcdn.com/bphoto/vFy-bw...,False,https://www.yelp.com/biz/rose-city-liquor-port...,56,"[{'alias': 'beer_and_wine', 'title': 'Beer, Wi...",4.5,"{'latitude': 45.5487873, 'longitude': -122.588...",[],$$,"{'address1': '7253 NE Sandy Blvd', 'address2':...",+15032847591,(503) 284-7591,9998.130687


In [22]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [23]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_vancouver_pho.csv.gz', compression='gzip',index=False)
 